# Pipeline ETL Simple

Este notebook demuestra cómo construir un pipeline ETL completo usando Python y Pandas.

**Referencia:** [Fundamentos Python](../fundamentos/fundamentos-python.md)

**Objetivos:**
- Extraer datos de CSV
- Transformar y limpiar datos
- Cargar a base de datos (opcional)
- Visualizar resultados

## 1. Importar librerías

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from pathlib import Path

# Cargar variables de entorno desde .env en la raíz del proyecto
env_path = Path().resolve().parent.parent / '.env'
if env_path.exists():
    load_dotenv(env_path)
    print("✅ Variables de entorno cargadas desde .env")
else:
    print("💡 No se encontró .env. Usando valores por defecto.")
    print("   Crea .env desde .env.example para configurar rutas y credenciales")

print("✅ Librerías importadas")

## 2. Extract: Cargar datos

In [ ]:
# Extract: Cargar datos
# En producción, cargarías desde archivo usando rutas del .env

# Obtener rutas desde variables de entorno o usar valores por defecto
data_source = os.getenv('DATA_SOURCE_PATH', '../data')
data_output = os.getenv('DATA_OUTPUT_PATH', '../data/output')

print(f"📁 Ruta de datos fuente: {data_source}")
print(f"📁 Ruta de datos procesados: {data_output}")

# Para este ejemplo, creamos datos de ejemplo
# En producción, usarías: df = pd.read_csv(os.path.join(data_source, 'ventas.csv'))
data = {
    'fecha': ['2024-01-15', '2024-01-16', '2024-01-17', '2024-01-18', '2024-01-19'],
    'producto': ['Producto A', 'Producto B', 'Producto A', 'Producto C', 'Producto B'],
    'cantidad': [5, 3, 2, 1, 4],
    'precio': [10.50, 25.00, 10.50, 50.00, 25.00],
    'cliente': ['Cliente 1', 'Cliente 2', 'Cliente 1', 'Cliente 3', 'Cliente 2']
}

df = pd.DataFrame(data)

print(f"\n✅ Extraídos {len(df)} registros")
print("\nPrimeras filas:")
df.head()

## 3. Transform: Limpiar y transformar

In [ ]:
# Copiar para no modificar original
df_clean = df.copy()

# Convertir fecha a datetime
df_clean['fecha'] = pd.to_datetime(df_clean['fecha'])

# Calcular total
df_clean['total'] = df_clean['cantidad'] * df_clean['precio']

# Eliminar duplicados
df_clean = df_clean.drop_duplicates()

# Eliminar filas con valores nulos críticos
df_clean = df_clean.dropna(subset=['fecha', 'producto', 'cantidad', 'precio'])

# Validar que cantidad y precio sean positivos
df_clean = df_clean[
    (df_clean['cantidad'] > 0) & 
    (df_clean['precio'] > 0)
]

print(f"✅ Transformados {len(df_clean)} registros")
print("\nDatos transformados:")
df_clean.head()

## 4. Validar transformaciones

In [ ]:
print("=== VALIDACIONES ===")
print(f"Registros originales: {len(df)}")
print(f"Registros después de transformación: {len(df_clean)}")
print(f"Registros eliminados: {len(df) - len(df_clean)}")
print(f"\nTotal calculado: €{df_clean['total'].sum():,.2f}")
print(f"Promedio por transacción: €{df_clean['total'].mean():,.2f}")

## 5. Load: Visualizar resultados (simulado)

In [ ]:
# Load: Guardar datos procesados
# Usar rutas desde variables de entorno

# Crear directorio de salida si no existe
os.makedirs(data_output, exist_ok=True)

# Guardar como Parquet (formato recomendado para Data Engineering)
output_file = os.path.join(data_output, 'ventas_procesadas.parquet')
df_clean.to_parquet(output_file, index=False)

print(f"✅ Datos guardados en: {output_file}")
print(f"\n💡 En producción, también podrías cargar a:")
print("  - PostgreSQL/MySQL (usando DB_HOST, DB_NAME del .env)")
print("  - Data Warehouse")
print("  - API (usando API_URL, API_KEY del .env)")
print("  - S3 (usando AWS_ACCESS_KEY_ID, S3_BUCKET_NAME del .env)")

## 6. Visualizar resultados

In [ ]:
# Ventas por producto
plt.figure(figsize=(10, 6))
ventas_producto = df_clean.groupby('producto')['total'].sum().sort_values(ascending=True)
ventas_producto.plot(kind='barh', color='steelblue')
plt.title('Ventas Totales por Producto')
plt.xlabel('Ventas Totales (€)')
plt.tight_layout()
plt.show()

In [ ]:
# Resumen del pipeline
print("=" * 60)
print("RESUMEN DEL PIPELINE ETL")
print("=" * 60)
print(f"\n✅ Extract: {len(df)} registros extraídos")
print(f"✅ Transform: {len(df_clean)} registros transformados")
print(f"✅ Total procesado: €{df_clean['total'].sum():,.2f}")
print(f"✅ Promedio: €{df_clean['total'].mean():,.2f}")
print("=" * 60)